### Import libraries
Import libraries, set parameters, and import classes

In [ ]:
import numpy as np
import tensorflow as tf
import pickle
np.random.seed(1)
tf.random.set_seed(1)
tf.keras.backend.set_floatx('float32')

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (3,3)
plt.rcParams.update({'font.size': 13})

In [ ]:
from __lib__.TANN_class import TANN
from __lib__.DataProcessing import PreProcessing

### Thermodynamics-based Artificial Neural Networks

Import trained model by specifying the file (filename) of the saved weights and the architecture.
The function import_compile(filename,input_shape) performs a check of the defined parameters wrt the saved model

In [ ]:
filename = './results/TANN_VonMises'

umat_dim = 4 # dimensional space

hidden_NN_Energy = 4 # number hidden layers in NN Energy
hidden_NN_Evolution = 3 # number of hidden layers in NN Evolution
material = 'inelastic'

TANNnet = TANN(umat_dim,
               material,
               hidden_NN_Energy,
               activation_NN_Energy='fast_gelu',
               hidden_NN_Evolution = hidden_NN_Evolution,
               activation_NN_Evolution = 'leaky_relu') # Initialize TANN class


# build model class
input_shape = [[None,umat_dim],[None,umat_dim],[None,umat_dim]]
TANNnet.build(input_shape)

# Compile model and import weights
TANNnet.import_compile(filename,input_shape)
TANNnet.summary()

### Inference - loading path (1)

Make predictions of the constitutive response under a biaxial loading path using the trained model. The loading path is predefined (see below), but you have to implement a function that call the trained model and make predictions.

Note. All quantities should be dimensional (not normalized)

In [ ]:
# Specify total number of increments and sub-increments (cylic loading)
increments = 30*5
sub_increments = increments//5

# Specify loading direction (0 = {}_11, 1 = {}_22)
umat_dir = [0,1]

# Specify strain increment magnitude
de = 2e-4

# Strain increment tensor (Voigt)
dstrain = np.zeros((increments,umat_dim))
dstrain[:sub_increments,umat_dir[0]] = de*np.ones((sub_increments))
dstrain[sub_increments:3*sub_increments,umat_dir[0]] = -de*np.ones((2*sub_increments))
dstrain[3*sub_increments:,umat_dir[0]] = de*np.ones((2*sub_increments))

dstrain[:,umat_dir[1]]=-dstrain[:,umat_dir[0]]

#### 1. Make predictions

Here goes your function to predict the stress and internal variables deriving from the deformation path specified by dstrain (see 3.1)

In [ ]:
h_strain,h_svars,h_stress = ???

#### 2. Import reference model solution

In [ ]:
file = './data/3D_PlaneStrain_VonMises_CSMA_inference_biaxial'
with open(file, 'rb') as f_obj:
    data = pickle.load(f_obj)

strain_ref,svars_ref,stress_ref,energy_ref,dissipationDot_ref = data

#### 3. Compare results

Plot the internal variables vs strain history as predicted by the model and compare it with the one of the reference model

In [ ]:
plt.plot(strain_ref[:,umat_dir[0]],svars_ref[:,umat_dir[0]],
         color='darkorange',
         alpha=1,linewidth=2, marker='s',markersize=0,label='ref')

plt.xlabel("$\\varepsilon_{11}$")
plt.ylabel("$z_{11}$")
plt.legend()
plt.show()

Plot the stress vs strain history as predicted by the model and compare it with the one of the reference model

In [ ]:
plt.plot(strain_ref[:,umat_dir[0]],stress_ref[:,umat_dir[0]],
         color='red',
         alpha=1,linewidth=1, marker='s',markersize=2,label='ref')

plt.xlabel("$\\varepsilon_{11}$")
plt.ylabel("$\sigma_{11}=\partial_{\\varepsilon_{11}} NN(\\varepsilon)$")
plt.legend()
plt.show()


Solution

In [ ]:
plt.plot(h_strain[:,umat_dir[0]],h_svars[:,umat_dir[0]],
         color = 'darkblue',
         alpha=1,linewidth=1, marker='o',markersize=4,label='TANN')

plt.plot(strain_ref[:,umat_dir[0]],svars_ref[:,umat_dir[0]],
         color='darkorange',
         alpha=1,linewidth=2, marker='s',markersize=0,label='ref')

plt.xlabel("$\\varepsilon_{11}$")
plt.ylabel("$z_{11}$")
plt.legend()
plt.show()

In [ ]:
plt.plot(h_strain[:,umat_dir[0]],h_stress[:,umat_dir[0]],
         alpha=1,linewidth=1, marker='o',markersize=5,label='TANN')

plt.plot(strain_ref[:,umat_dir[0]],stress_ref[:,umat_dir[0]],
         color='red',
         alpha=1,linewidth=1, marker='s',markersize=2,label='ref')

plt.xlabel("$\\varepsilon_{11}$")
plt.ylabel("$\sigma_{11}=\partial_{\\varepsilon_{11}} NN(\\varepsilon)$")
plt.legend()
plt.show()

### Inference - loading path (2)
Make predictions of the constitutive response under a simple shear loading path (direction 12) using the trained model. The loading path is predefined (see 1. Loading path). Use the function that you implemented above.

#### 1. Loading path

In [ ]:
increments = 600

# Specify loading direction (-1 = {}_12)
umat_dir = -1

# Strain increment
de = 2e-4

# Strain increment tensor (Voigt)
dstrain = np.zeros((increments,umat_dim))
dstrain[:increments,umat_dir] = 2*de*np.ones((increments))

#### 2. Make predictions

In [ ]:
h_strain,h_svars,h_stress = ???

#### 3. Import reference model solution

In [ ]:
file = './data/3D_PlaneStrain_VonMises_CSMA_inference_shear'
with open(file, 'rb') as f_obj:
    data = pickle.load(f_obj)
strain_ref,svars_ref,stress_ref,energy_ref,dissipationDot_ref = data

#### 4. Compare results

In [ ]:
plt.plot(hstrain[:,umat_dir],
         hsvars[:,umat_dir],
         color = 'blue',
         alpha=1,linewidth=1, marker='o',markersize=4,label='TANN')
plt.plot(strain_ref[:,umat_dir],
         svars_ref[:,umat_dir],
         color='red',
         alpha=1,linewidth=1, marker='s',markersize=2,label='ref')
plt.xlabel("$\\varepsilon_{12}$")
plt.ylabel("$z_{12}$")
plt.legend()
plt.show()

In [ ]:
plt.plot(hstrain[:,umat_dir],
         hstress[:,umat_dir],
         alpha=1,linewidth=1, marker='o',markersize=4,label='TANN')
plt.plot(strain_ref[:,umat_dir],
         stress_ref[:,umat_dir],
         color='red',
         alpha=1,linewidth=2, marker='s',markersize=0,label='ref')
plt.xlabel("$\\varepsilon_{12}$")
plt.ylabel("$\sigma_{12}=\partial_{\\varepsilon_{12}} NN(\\varepsilon)$")
plt.legend()
plt.show()


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b33f54f1-f98d-4fed-99c0-2484f115edd9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>